<a href="https://colab.research.google.com/github/srenna/RAGOpenAI/blob/main/RAG_with_OpenAI_and_Azure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
!pip install langchain --progress-bar off
!pip install openai --progress-bar off
!pip install tiktoken --progress-bar off
!pip install faiss-gpu --progress-bar off
!pip install langchain_experimental --progress-bar off
!pip install "langchain[docarray]" --progress-bar off
!pip install -qqq pypdf==3.15.0 --progress-bar off
!pip install -U langchain-openai --progress-bar off

  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib:

In [5]:
from langchain.chains import RetrievalQA
# from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken
from langchain_openai import ChatOpenAI
import os
import re
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chat_models import AzureChatOpenAI
import openai

In [ ]:
# Prompt the user for their OpenAI API key
api_key = input("Please enter your OpenAI API key: ")

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = api_key

# Optionally, check that the environment variable was set correctly
print("OPENAI_API_KEY has been set!")


In [7]:
llm_model = "gpt-4.0-turbo"

In [8]:
# load the directory
pdf_file_path = '/content/drive/MyDrive/POVs/Test PDF'
loader = PyPDFDirectoryLoader(pdf_file_path)
data = loader.load()

print(len(data))
# chunk the data
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
data = text_splitter.split_documents(data)

16


In [9]:
# preview some data
data

[Document(page_content='An integrated approach to \ncombat cyber risk\nSecuring industrial operations \nin\xa0oil and gas', metadata={'source': '/content/drive/MyDrive/POVs/Test PDF/intergrated-approach-combat-cyber-risk-oil-gas.pdf', 'page': 0}),
 Document(page_content='ContentsForeword  1\nIntroduction  2\nUnderstanding the risks  4\nConduct a\xa0maturity assessment  6\nBuild a\xa0unified program  8\nImplement key controls  9\nEmbrace good governance  10\nConclusion  11\nContact us  12', metadata={'source': '/content/drive/MyDrive/POVs/Test PDF/intergrated-approach-combat-cyber-risk-oil-gas.pdf', 'page': 1}),
 Document(page_content='Foreword\nThe oil and gas industry is moving into the \nnext stage of evolution, whereby robotics, \ndigitization, and the Internet of Things \n(IoT) are rapidly being integrated into the \noperational environment. The\xa0interest of \ncyber criminals in industrial operations has \nincreased over the last decade resulting in \ncyberattacks that have compr

In [10]:
# Define a function to clean the text
def clean_text(text):
    # Remove '\xa0' (non-breaking space)
    text = re.sub(r'\xa0', ' ', text)

    # Remove '\n' (newlines)
    text = re.sub(r'\n', ' ', text)

    # Remove consecutive dots ('.....') with just one dot
    text = re.sub(r'\.{2,}', '', text)

    # Remove '\uf0b7'
    text = re.sub(r'\uf0b7', ' ', text)

    # Remove consecutive spaces
    text = re.sub(r' +', ' ', text)

    return text

# Clean the text in the 'page_content' of each document in the 'data' list
import re

for document in data:
    document.page_content = clean_text(document.page_content)

In [11]:
data

[Document(page_content='An integrated approach to combat cyber risk Securing industrial operations in oil and gas', metadata={'source': '/content/drive/MyDrive/POVs/Test PDF/intergrated-approach-combat-cyber-risk-oil-gas.pdf', 'page': 0}),
 Document(page_content='ContentsForeword 1 Introduction 2 Understanding the risks 4 Conduct a maturity assessment 6 Build a unified program 8 Implement key controls 9 Embrace good governance 10 Conclusion 11 Contact us 12', metadata={'source': '/content/drive/MyDrive/POVs/Test PDF/intergrated-approach-combat-cyber-risk-oil-gas.pdf', 'page': 1}),
 Document(page_content='Foreword The oil and gas industry is moving into the next stage of evolution, whereby robotics, digitization, and the Internet of Things (IoT) are rapidly being integrated into the operational environment. The interest of cyber criminals in industrial operations has increased over the last decade resulting in cyberattacks that have compromised both production and safety. These attacks 

In [12]:
# !pip install -qqq chromadb==0.4.5 --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.


In [18]:
from langchain_openai import OpenAIEmbeddings

In [19]:
# Create vector store
embeddings = OpenAIEmbeddings()
#vectorstore = Chroma.from_documents(data, embeddings)
vectorstore = FAISS.from_documents(data, embedding=embeddings)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: ea5dac28********************4f35. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
# # Create conversation chain
# llm = ChatOpenAI(temperature=0.7, model_name="gpt-4")
# memory = ConversationBufferMemory(
# memory_key='chat_history', return_messages=True)
# conversation_chain = ConversationalRetrievalChain.from_llm(
#         llm=llm,
#         chain_type="stuff",
#         retriever=vectorstore.as_retriever(),
#         memory=memory
#         )

In [ ]:
llm = AzureChatOpenAI(deployment_name="GPT4",
                      model_name="gpt-4",
                      openai_api_base="https://genai-gpt.openai.azure.com/",
                      openai_api_version="2023-07-01-preview",
                      openai_api_key="ea5dac28837a4a91bca6367973174f35",
                      openai_api_type="azure",
                      temperature=0.5)

In [ ]:
memory = ConversationBufferMemory(

memory_key='chat_history', return_messages=True)

conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        memory=memory
        )

In [ ]:
query = "Explain the integrated approach to combat cyber risk proposed by Deloitte."
result = conversation_chain({"question": query})
answer = result["answer"]
answer

In [ ]:
query = "What are the upstream cyberthreats?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

In [ ]:
query = "Who are the authors of this document?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

In [ ]:
query = "Who are the threat actors"
result = conversation_chain({"question": query})
answer = result["answer"]
answer